In [120]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as	plt
import seaborn as sns
import json

In [75]:
df = pd.read_csv('train.txt', sep=' ', error_bad_lines=False, engine="python", quoting=3,  dtype=str, encoding='utf-8', names=["Token", "Label"])

C:\Users\Ephyrus\AppData\Local\Temp\ipykernel_26988\2090610779.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv('train.txt', sep=' ', error_bad_lines=False, engine="python", quoting=3,  dtype=str, encoding='utf-8', names=["Token", "Label"])


In [76]:
display(df)

,Token,Label
0,Estuvimos,O
1,hace,O
2,poco,O
3,mi,O
4,pareja,O
...,...,...
31171,junto,O
31172,con,O
31173,la,O
31174,carta,O


In [77]:
df_token = df.loc[:,["Token"]]
df_label = df.loc[:,["Label"]]

In [78]:
y_all = pd.DataFrame(df_label.value_counts()).reset_index()
y_all.columns = ['Label', 'Count']
y_all["Location"] = [np.where((df_label.squeeze()).to_numpy() == i)[0] for i in df_label.squeeze().unique()]

x_all = pd.DataFrame(df_token.value_counts()).reset_index()
x_all.columns = ['Token', 'Count']
x_all = x_all.drop("Count", axis=1)

# Need to check more, this takes >1m to do
new_x_all = pd.DataFrame({'Token': np.repeat(x_all['Token'], 7)}).reset_index()
new_x_all.columns = ['Index', 'Token']
new_x_all["Label"] = np.tile(y_all.loc[:, "Label"].unique(), int(len(new_x_all)/7))
count_x_all = []
location_x_all = []
for i in new_x_all.loc[:, "Token"].unique():
    for j in new_x_all.loc[:, "Label"].unique():
        cond_1 = df["Token"] == i
        cond_2 = df["Label"] == j
        df_ij = df[cond_1 & cond_2]
        count_x_all += [len(df_ij)]
        location_x_all += [df_ij.index.to_list()]
new_x_all["Count"] = count_x_all
new_x_all["Location"] = location_x_all

print(len(new_x_all))
print(type(new_x_all))
display(new_x_all)

print(len(y_all))
print(type(y_all))
display(y_all)

35280
<class 'pandas.core.frame.DataFrame'>


,Index,Token,Label,Count,Location
0,0,",",O,1664,"[13, 17, 20, 23, 36, 65, 85, 89, 102, 128, 132..."
1,0,",",B-positive,0,[]
2,0,",",B-negative,0,[]
3,0,",",I-positive,2,"[5985, 5989]"
4,0,",",I-negative,3,"[12379, 20294, 28914]"
...,...,...,...,...,...
35275,5039,evitar,B-negative,0,[]
35276,5039,evitar,I-positive,0,[]
35277,5039,evitar,I-negative,0,[]
35278,5039,evitar,B-neutral,0,[]


7
<class 'pandas.core.frame.DataFrame'>


,Label,Count,Location
0,O,29035,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
1,B-positive,1160,"[16, 19, 22, 25, 60, 63, 75, 86, 90, 113, 204,..."
2,B-negative,381,"[39, 98, 124, 131, 133, 137, 323, 382, 472, 62..."
3,I-positive,314,"[153, 591, 685, 1053, 1180, 1467, 1973, 1994, ..."
4,I-negative,171,"[154, 155, 156, 157, 158, 159, 160, 161, 162, ..."
5,B-neutral,72,"[313, 314, 315, 937, 1400, 1401, 1939, 2054, 2..."
6,I-neutral,43,"[802, 1171, 1626, 1627, 1724, 1725, 1726, 1727..."


$1. \:\:\: e(x \mid y)=\frac{\operatorname{Count}(y \rightarrow x)}{\operatorname{Count}(y)}$

In [84]:
def emm_par(x, y):
    # x is an instance of Token
    # y is an instance of Label
    cond_1 = new_x_all["Token"] == x
    cond_2 = new_x_all["Label"] == y
    num = ((new_x_all[cond_1 & cond_2])["Count"]).values[0]
    cond_3 = y_all["Label"] == y
    denom = ((y_all[cond_3])["Count"]).values[0]
    return num/denom

In [86]:
a = emm_par(",", "I-negative")
print(a)

0.017543859649122806


$2. \:\:\:e(x \mid y)=\left\{\begin{array}{cc}
\frac{\operatorname{count}(y \rightarrow x)}{\operatorname{Count}(y)+k} & \text { If the word token } x \text { appears in the training set } \\
\frac{k}{\operatorname{Count}(y)+k} & \text { If word token } x \text { is the special token \#UNK\# }
\end{array}\right.$

In [92]:
def emm_par_test(x, y):
    # x is an instance of Token
    # y is an instance of Label
    k = 1
    if x in new_x_all.loc[:, "Token"].unique():
        cond_1 = new_x_all["Token"] == x
        cond_2 = new_x_all["Label"] == y
        num = ((new_x_all[cond_1 & cond_2])["Count"]).values[0]
    else:
        num = k
    cond_3 = y_all["Label"] == y
    denom = ((y_all[cond_3])["Count"]).values[0] + k
    return num/denom

In [93]:
a = emm_par_test(",", "I-negative")
print(a)

b = emm_par_test("Kontol", "I-neutral")
print(b)

0.01744186046511628
0.022727272727272728


$3. \:\:\:y^*=\underset{y}{\arg \max } \:e(x \mid y)$

In [105]:
df_y = pd.DataFrame()

emm_par_score_all = []
y_star_all = []
token_all = new_x_all.loc[:, "Token"].unique()
label_all = new_x_all.loc[:, "Label"].unique()

In [106]:
# Need to check more, this takes >2m to do
for i in token_all:
    emm_par_score = 0
    y_star = ""
    for j in label_all:
        emm_par_score_temp = emm_par(i,j)
        if emm_par_score_temp > emm_par_score:
            emm_par_score = emm_par_score_temp
            y_star = j
        else:
            pass
    y_star_all += [y_star]
    emm_par_score_all += [emm_par_score]

In [109]:
print(token_all)

[',' '.' 'de' ... 'escasamente' 'escasez' 'evitar']


In [116]:
print(emm_par_score_all)
print(y_star_all)

[0.05730816916930707, 0.055896128943380634, 0.18095238095238095, 0.0352665656426505, 0.029101804656288744, 0.0380952380952381, 0.022727272727272728, 0.022727272727272728, 0.045454545454545456, 0.013638242182118749, 0.012708362033337925, 0.012570602011296322, 0.09090909090909091, 0.010469761675161868, 0.009298801487808239, 0.00864444138311062, 0.008506681361069017, 0.007886761261881802, 0.14556416881998277, 0.022727272727272728, 0.006578041052486569, 0.10508182601205857, 0.005682600909216145, 0.00547596087615374, 0.00544152087064334, 0.005372640859622538, 0.04712041884816754, 0.006349206349206349, 0.005097120815539331, 0.005028240804518529, 0.0049938007990081275, 0.00468384074941452, 0.004511640721862516, 0.004511640721862516, 0.005813953488372093, 0.023255813953488372, 0.004236120677779309, 0.0136986301369863, 0.022727272727272728, 0.005813953488372093, 0.0030651604904256783, 0.022727272727272728, 0.002927400468384075, 0.005813953488372093, 0.002651880424300868, 0.002583000413280066, 0

In [117]:
df_y["Token"] = token_all
df_y["y*"] = y_star_all
df_y["Max e(x|y)"] = emm_par_score_all

display(df_y)

,Token,Max e(x|y),y*
0,",",0.057308,O
1,.,0.055896,O
2,de,0.180952,I-positive
3,y,0.035267,O
4,que,0.029102,O
...,...,...,...
5035,escalibadas,0.000861,B-positive
5036,JoséHe,0.000034,O
5037,escasamente,0.000034,O
5038,escasez,0.000034,O


In [122]:
df_y_dict = {row['Token']: (row['y*'], row['Max e(x|y)']) for row in df_y.to_dict(orient='records')}
print(df_y_dict)

{',': ('O', 0.05730816916930707), '.': ('O', 0.055896128943380634), 'de': ('I-positive', 0.18095238095238095), 'y': ('O', 0.0352665656426505), 'que': ('O', 0.029101804656288744), 'la': ('I-positive', 0.0380952380952381), 'el': ('I-neutral', 0.022727272727272728), 'en': ('I-neutral', 0.022727272727272728), 'a': ('I-neutral', 0.045454545454545456), 'muy': ('O', 0.013638242182118749), 'no': ('O', 0.012708362033337925), 'un': ('O', 0.012570602011296322), 'con': ('I-neutral', 0.09090909090909091), 'es': ('O', 0.010469761675161868), 'lo': ('O', 0.009298801487808239), 'por': ('O', 0.00864444138311062), 'los': ('O', 0.008506681361069017), 'una': ('O', 0.007886761261881802), 'comida': ('B-positive', 0.14556416881998277), 'pero': ('I-neutral', 0.022727272727272728), 'para': ('O', 0.006578041052486569), 'servicio': ('B-positive', 0.10508182601205857), 'bien': ('O', 0.005682600909216145), 'se': ('O', 0.00547596087615374), '!': ('O', 0.00544152087064334), 'El': ('O', 0.005372640859622538), 'restaur

Testing

In [125]:
df_test = pd.read_csv('dev_out.txt', sep=' ', error_bad_lines=False, engine="python", quoting=3,  dtype=str, encoding='utf-8', names=["Token", "Label"])

C:\Users\Ephyrus\AppData\Local\Temp\ipykernel_26988\2761620611.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df_test = pd.read_csv('dev_out.txt', sep=' ', error_bad_lines=False, engine="python", quoting=3,  dtype=str, encoding='utf-8', names=["Token", "Label"])


In [126]:
display(df_test)

,Token,Label
0,Plato,B-positive
1,degustación,I-positive
2,:,O
3,un,O
4,poco,O
...,...,...
4307,sin,O
4308,dejarte,O
4309,la,O
4310,cartera,O
